# Recommender : Model 2

In the previous notebook, we had built model 1 and the scores used for recommendations does not seem ideal. 

In this notebook, we will build model 2, taking in `genre`, `categories`, `language`, `description` and `weighted average`. 

---

## Import Libraries

In this section, we will import all the libraries that will be used in this notebook.

In [1]:
# For Calculation and Data Manipulation
import numpy as np
import pandas as pd
import math

# for data visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# for cosine similarity calculation
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.preprocessing import StandardScaler

# for Approximate Nearest Neighbor
import hnswlib

# for NLP
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords
from stopwordsiso import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re, string

# For file exportion folder creation
import os

# for datetime conversion
import datetime

# for data 
import sqlite3
import h5py

# import created functions
from utils import get_recommendations, fit_hnsw_index

# this setting widens how many characters pandas will display in a column:
pd.options.display.max_colwidth = 500

# this setting allows us to see up to 50 columns
pd.options.display.max_columns = 50

---

## Functions

In this section, we will list down all the functions that are being used in the notebook as a summary. The functions can be found in [utils.py](./utils.py).

1. `get_recommendations` : get top 10 recommendations based on cosine similarity
2. `fit_hnsw_index` : get top 10 recommendations using approximate nearest neighbours calculated using cosine similarity

---

## Read data file

First, we will connect to the database. 

In [2]:
# connecting to DB file
con = sqlite3.connect('../data/steam_db.db')

In [3]:
# ensure that connection is establish
sql_query = '''
SELECT *
FROM main
LIMIT 5
'''

pd.read_sql(sql_query, con)

,steam_appid,name,release_date,type,developer,publisher,num_packages
0,10.0,Counter-Strike,2000-11-01 00:00:00,game,Valve,Valve,2
1,20.0,Team Fortress Classic,1999-04-01 00:00:00,game,Valve,Valve,1
2,30.0,Day of Defeat,2003-05-01 00:00:00,game,Valve,Valve,1
3,40.0,Deathmatch Classic,2001-06-01 00:00:00,game,Valve,Valve,1
4,50.0,Half-Life: Opposing Force,1999-11-01 00:00:00,game,Gearbox Software,Valve,1


From our EDA, we know that we have the below tables: 

1. main
2. genre
3. genre_mapping
4. categories
5. categories_mapping
6. description
7. price
8. statistics
9. media
10. requirements
11. tag
12. language
13. support_info

---

## Data Extraction

### Weighted Average
We will use the weighted average that was previously created for the simple recommender exploration, with the Mathematical formula as follows:

$ Weighted Average(WA) =  (\frac{1}{6} \cdot F) + (\frac{2}{6} \cdot O) + (\frac{3}{6} \cdot P)$

where 
- P is the percentage positive review: ($\frac{positive}{positive+negative}$) with the weight of 3 as this is the number of game reviews
- O is the midpoint estimate of number of owners: ($\frac{max_owners + min_owners}{2}$) with the weight of 1 as this is an estimate of number of owners
- F is the average_forever: with the weight of 2 as this is the average playtime since March 2009 in minutes. 

In [4]:
# function to calculate weighted review
def weighted_review(x):
    P = x['percentage_positive']
    O = x['midpt_est_owners']
    F = x['average_forever']
    
    # calculation based on formula
    return ((1/6 * F) + (2/6 * O) + (3/6 * P))

In [5]:
# create dataframe for the calculation
sql_query = """
SELECT * 
FROM statistics
"""

df_stat = pd.read_sql(sql_query, con)

In [6]:
# create percentage_postive
df_stat['percentage_positive'] = df_stat['positive'] / (df_stat['positive'] + df_stat['negative'])

# create midpt_est_owners
df_stat['midpt_est_owners'] = (df_stat['max_owners'] + df_stat['min_owners']) / 2

# create weighted average
df_stat['wa'] = df_stat.apply(weighted_review, axis=1)

# fill in missing values
df_stat['wa'].fillna(0, inplace=True)

In [7]:
# see statistics of weighted average
df_stat[['wa']].describe()

,wa
count,4.901500e+04
mean,4.400518e+04
std,4.364608e+05
min,0.000000e+00
25%,3.333683e+03
50%,3.333817e+03
75%,1.166712e+04
max,5.000632e+07


We see that the data is left skewed. We will need to scale the data before using for calculation. 

### Description

We will look at the description and look to take top 3000 words.We will look at the description and look to take top 3000 words.

In [8]:
# import columns required
sql_query = '''
SELECT steam_appid, detailed_description
FROM description
'''

df_des = pd.read_sql(sql_query, con)

In [9]:
# create list of stopwords
final_stopwords = stopwords(["en", "ja", "ko", "zh"])

In [10]:
# instantiate CountVectorizer
cv = CountVectorizer(stop_words=final_stopwords, max_features=3_000)

# fit and transform the column
transformed_cv = cv.fit_transform(df_des['detailed_description'])

# convert transformed data to dataframe
matrix_cv = transformed_cv.todense()   # converts to matrix
df_cv_words = pd.DataFrame(matrix_cv, columns=cv.get_feature_names_out())

C:\Users\Mitchelle\anaconda3\envs\ga-github\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan', '가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '미치다', '바꾸어서', '바꿔', '밖에', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것

In [11]:
# see shape and first 5 rows
print(df_cv_words.shape)
df_cv_words.head()

(49015, 3000)


,000,01,02,100,1000,11,12,120,13,14,15,150,16,1620520,17,18,19,20,200,2014,2015,2016,2017,2018,2019,...,woods,word,workers,workshop,worlds,worldwide,worry,worse,worst,worth,worthy,write,writing,written,wrong,xbox,xp,yellow,york,youtube,zombie,zombies,zone,zones,zoom
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
# update the df to include steam_appid
df_cv_words['steam_appid'] = df_des['steam_appid']

## Combining the data

With the data extracted from the description (top 2000) and weighted average, we will now extract the remaining data and use it for the calculation. 

In [13]:
# create columns required
sql_query = '''
SELECT steam_appid, name, developer
FROM main
'''

df_main = pd.read_sql(sql_query, con)

In [14]:
sql_query = '''
SELECT *
FROM genre
'''

df_genre = pd.read_sql(sql_query, con)

In [15]:
sql_query = '''
SELECT *
FROM categories
'''

df_categories = pd.read_sql(sql_query, con)

In [16]:
sql_query = '''
SELECT *
FROM language
'''

df_language = pd.read_sql(sql_query, con)

### Managing the data: `genre`, `categories`, `language`, `tag`, `weighted average`, `description`

We will combine the different tables into 1 for our calculations, but first we will re-arrange the data that is being read out from the column. 

In [17]:
# drop columns that are not required
df_genre = df_genre.drop(columns=["genre_id", "genre"])
df_categories = df_categories.drop(columns=['categories_id','categories_description'])
df_language = df_language.drop(columns=['languages'])

# rename column in language
df_language.rename(columns = {col: (col+"_lang") for col in df_language.columns if col != 'steam_appid'}, inplace=True)

### Model 2: `genre`, `categories`, `language`, `weighted average`, `description`

We will build the model using 
- `genre`: game genre
- `categories`: game categories
- `language`: language of the game
- `weighted average`: feature created using `statistics` table
- `description`: Top 2000 words

`tag` is not included in model 2 calculation as concluded in model 1 evaluation, `tag` seems to be creating more noise which is created by users who will select based on their experience. 

In [18]:
# create df of model 2
df_model_two = df_genre.copy()
df_model_two = df_model_two.join(df_categories.set_index("steam_appid"), on="steam_appid")
df_model_two = df_model_two.join(df_language.set_index("steam_appid"), on="steam_appid")
df_model_two = df_model_two.join(df_stat[['steam_appid','wa']].set_index("steam_appid"), on="steam_appid")
df_model_two = df_model_two.join(df_cv_words.set_index("steam_appid"), on="steam_appid")

# drop the column used for undex setting
df_model_two = df_model_two.drop(columns=['steam_appid'])

In [19]:
# see shape and first 2 rows
print(df_model_two.shape)
df_model_two.head(2)

(49015, 3103)


,genre_id_1,genre_id_18,genre_id_2,genre_id_23,genre_id_25,genre_id_28,genre_id_29,genre_id_3,genre_id_37,genre_id_4,genre_id_50,genre_id_51,genre_id_52,genre_id_53,genre_id_54,genre_id_55,genre_id_56,genre_id_57,genre_id_58,genre_id_59,genre_id_60,genre_id_70,genre_id_71,genre_id_72,genre_id_73,...,woods,word,workers,workshop,worlds,worldwide,worry,worse,worst,worth,worthy,write,writing,written,wrong,xbox,xp,yellow,york,youtube,zombie,zombies,zone,zones,zoom
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
# standstardscale df_model_one
df_model_two = StandardScaler().fit_transform(df_model_two)

---

## Cosine Similarity

Let us build a model using cosine similarity. 

In [23]:
%%time

# find the cosine similarity 
cos_sim_two = cosine_similarity(df_model_two, df_model_two)

Wall time: 4min 9s


In [24]:
# create reverse mapping of name and index
indices = pd.Series(df_main.index, index=df_main['name'])

We will test the recommender against 4 games:
1. `Half-Life 2: Lost Coast`
2. `Counter-Strike`
3. `Assetto Corsa`
4. `Kenshi`

In [23]:
# get recommendations for Half-Life 2: Lost Coast
get_recommendations(df_main, indices, "Half-Life 2: Lost Coast", cos_sim_two)

,name,cos_sim
13123,PUBG: BATTLEGROUNDS,0.618633
22,Dota 2,0.602086
19,Team Fortress 2,0.571677
1247,Terraria,0.521267
25,Counter-Strike: Global Offensive,0.512704
8,Half-Life: Blue Shift,0.467606
17754,RedEyes 赤瞳之勋,0.348416
30191,Elpida: Crônicas de uma guerreira,0.345356
38517,大千世界,0.340297
24901,梦本无忧,0.329289


In [24]:
# get recommendations for Counter-Strike
get_recommendations(df_main, indices, "Counter-Strike", cos_sim_two)

,name,cos_sim
1,Team Fortress Classic,0.642793
3,Deathmatch Classic,0.564204
15,Half-Life Deathmatch: Source,0.536008
5,Ricochet,0.522276
6,Half-Life,0.509510
1348,Call of Duty®: Black Ops II,0.461331
2,Day of Defeat,0.447630
4,Half-Life: Opposing Force,0.447143
7736,DiRT 4,0.445712
7,Counter-Strike: Condition Zero,0.440548


In [25]:
# get recommendations for Assetto Corsa
get_recommendations(df_main, indices, "Assetto Corsa", cos_sim_two)

,name,cos_sim
8047,Automobilista,0.299717
25737,RDS - The Official Drift Videogame,0.271131
4518,VRC PRO,0.267158
21236,Assetto Corsa Competizione,0.257612
2587,iRacing,0.248591
7489,Drift Streets Japan,0.232229
11032,DRIFT21,0.229881
24833,GRITS Racing,0.226602
2523,Victory: The Age of Racing,0.225300
30109,Automobilista 2,0.224344


In [26]:
# get recommendations for Kenshi
get_recommendations(df_main, indices, "Kenshi", cos_sim_two)

,name,cos_sim
28227,Nomads of the Fallen Star,0.170223
1714,Gnomoria,0.153603
41722,Arid,0.140227
24685,Amber's Airline - High Hopes,0.140176
28120,Space Digger,0.139804
13785,Survivalizm - The Animal Simulator,0.138259
32659,Adam: Robot World,0.138020
8525,Expeditions: Viking,0.134360
30720,Orders Of The Ruler,0.133270
7371,No1Left,0.132102


--- 

## Approximate Nearest Neighbors (ANN) using `Hnswlib`

We will build a model using Hnswlib to calculate the distance based on squared L2 (least square error), as we have calculated cosine similarity above.

In [27]:
%%time

# create model
# model from https://pub.towardsai.net/knn-k-nearest-neighbors-is-dead-fc16507eb3e?sk=b964df6dccf263518b244d4264ba088d
p = fit_hnsw_index(df_model_two)

# set k as 11 to get 10 recommendations
ann_neighbor_indices, ann_distances = p.knn_query(df_model_two, k=11)

Wall time: 31min 38s


In [28]:
# get recommendation for Half-Life 2: Lost Coast
df_main.loc[list(ann_neighbor_indices[list(df_main['name']).index("Half-Life 2: Lost Coast")]), :]

,steam_appid,name,developer
14,340.0,Half-Life 2: Lost Coast,Valve
8,130.0,Half-Life: Blue Shift,Gearbox Software
17754,711710.0,RedEyes 赤瞳之勋,RiceMaster
30191,1069290.0,Elpida: Crônicas de uma guerreira,Daniel Pazos
4397,331470.0,Everlasting Summer,Soviet Games
1247,105600.0,Terraria,Re-Logic
38517,1348300.0,大千世界,滑稽工作室
24901,909020.0,梦本无忧,幻想禁
1564,214700.0,Thirty Flights of Loving,Blendo Games
42443,1492870.0,堕星之乱,龙骨工作室


In [29]:
# get recommendation for Counter-Strike
df_main.loc[list(ann_neighbor_indices[list(df_main['name']).index("Counter-Strike")]), :]

,steam_appid,name,developer
0,10.0,Counter-Strike,Valve
1,20.0,Team Fortress Classic,Valve
3,40.0,Deathmatch Classic,Valve
15,360.0,Half-Life Deathmatch: Source,Valve
6,70.0,Half-Life,Valve
5,60.0,Ricochet,Valve
1348,202970.0,Call of Duty®: Black Ops II,Treyarch
40105,1402480.0,NARAKA: BLADEPOINT / 永劫無間 - Test Server,24 Entertainment
7736,421020.0,DiRT 4,"Codemasters, Feral Interactive (Mac), Feral Interactive (Linux)"
4397,331470.0,Everlasting Summer,Soviet Games


In [30]:
# get recommendation for Assetto Corsa
df_main.loc[list(ann_neighbor_indices[list(df_main['name']).index("Assetto Corsa")]), :]

,steam_appid,name,developer
20102,775900.0,MotoGP™18,Milestone S.r.l.
37542,1308140.0,Drive & Drift,UnknownStudio
20698,791150.0,EV3 - Drag Racing,KABloom Interactive
39243,1374010.0,Grand Prix Racing On Line,Pix Arts
13932,601770.0,Sparc,CCP
44793,1587610.0,Little drift,Vitok
45853,1644930.0,Fastlane Bowling,LAB Rats Game
32762,1149120.0,Drift 1969,Polantronic
45222,1607640.0,Arcade Drift,EFatullaev
49013,1885640.0,Warehouse Simulator: Forklift Driver,"Maks Volegov, Andreev Worlds"


In [31]:
# get recommendation for Kenshi
df_main.loc[list(ann_neighbor_indices[list(df_main['name']).index("Kenshi")]), :]

,steam_appid,name,developer
46632,1693360.0,Dark Forest Project,Phoenixxx Games
9844,489370.0,Quarantine,Sproing
49008,1883580.0,Cactus Simulator,Iful GS
15086,641950.0,Last Stonelord,Morganti Livio
27831,999250.0,Chucky,Shichifu
42443,1492870.0,堕星之乱,龙骨工作室
48191,1798130.0,ReTox,Dopamine Games
28503,1018800.0,DEEEER Simulator: Your Average Everyday Deer Game,Gibier Games
10304,501450.0,Puzzle Cross,Ayy Caramba Games
45980,1653100.0,Heliotropism,Verde


Comparing the two, we see that the recommender built by both models are similar to each other. 



---

## Analysis

For our analysis, we will use the cosine similarity model which is more interpretable as compared to `Hnswlib`.

We will use `Counter-Strike` for our analysis.

In [32]:
# get recommendations for Counter-Strike
get_recommendations(df_main, indices, "Counter-Strike", cos_sim_two)

,name,cos_sim
1,Team Fortress Classic,0.642793
3,Deathmatch Classic,0.564204
15,Half-Life Deathmatch: Source,0.536008
5,Ricochet,0.522276
6,Half-Life,0.509510
1348,Call of Duty®: Black Ops II,0.461331
2,Day of Defeat,0.447630
4,Half-Life: Opposing Force,0.447143
7736,DiRT 4,0.445712
7,Counter-Strike: Condition Zero,0.440548


First, we will look at the top 3 games and their recommendations

In [33]:
get_recommendations(df_main, indices, "Team Fortress Classic", cos_sim_two)

,name,cos_sim
3,Deathmatch Classic,0.688236
0,Counter-Strike,0.642793
15,Half-Life Deathmatch: Source,0.635566
6,Half-Life,0.628080
5,Ricochet,0.627148
1348,Call of Duty®: Black Ops II,0.566292
7736,DiRT 4,0.546883
4,Half-Life: Opposing Force,0.533578
7,Counter-Strike: Condition Zero,0.514653
2,Day of Defeat,0.485031


In [34]:
get_recommendations(df_main, indices, "Deathmatch Classic", cos_sim_two)

,name,cos_sim
5,Ricochet,0.725354
1,Team Fortress Classic,0.688236
15,Half-Life Deathmatch: Source,0.664335
6,Half-Life,0.620387
0,Counter-Strike,0.564204
7736,DiRT 4,0.555257
1348,Call of Duty®: Black Ops II,0.531683
4,Half-Life: Opposing Force,0.524549
7,Counter-Strike: Condition Zero,0.509289
2,Day of Defeat,0.471427


In [35]:
get_recommendations(df_main, indices, "Half-Life Deathmatch: Source", cos_sim_two)

,name,cos_sim
3,Deathmatch Classic,0.664335
1,Team Fortress Classic,0.635566
5,Ricochet,0.599928
6,Half-Life,0.584650
1348,Call of Duty®: Black Ops II,0.560591
4,Half-Life: Opposing Force,0.543661
7736,DiRT 4,0.542822
0,Counter-Strike,0.536008
7,Counter-Strike: Condition Zero,0.512017
13,Half-Life 2: Deathmatch,0.499890


We see that for the top 3 games recommended for `Counter-Strike` has similar recommendations. Other than for `Half-Life Deathmatch: Source`, these 4 games are top 3 recommendations of each other. Let us get the data for these 4 games:
1) `Counter-Strike`
2) `Team Fortress Classic`
3) `Deathmatch Classic`
4) `Half-Life Deathmatch: Source`

### `main`

In [36]:
# see the information in main table
sql_query = """
SELECT * 
FROM main
WHERE (name = 'Counter-Strike') OR (name = 'Team Fortress Classic') OR (name = 'Deathmatch Classic') OR (name = 'Half-Life Deathmatch: Source')
"""

pd.read_sql(sql_query, con)

,steam_appid,name,release_date,type,developer,publisher,num_packages
0,10.0,Counter-Strike,2000-11-01 00:00:00,game,Valve,Valve,2
1,20.0,Team Fortress Classic,1999-04-01 00:00:00,game,Valve,Valve,1
2,40.0,Deathmatch Classic,2001-06-01 00:00:00,game,Valve,Valve,1
3,360.0,Half-Life Deathmatch: Source,2006-05-01 00:00:00,game,Valve,Valve,1


We see that the games are made by the same developer and publisher. 

### `description`

In [37]:
# see the information from description
sql_query = """
SELECT * 
FROM description
WHERE steam_appid IN (SELECT steam_appid FROM main
                        WHERE (name = 'Counter-Strike') OR (name = 'Team Fortress Classic') OR (name = 'Deathmatch Classic') OR (name = 'Half-Life Deathmatch: Source'))
"""

pd.read_sql(sql_query, con)

,steam_appid,about_the_game,background,detailed_description,short_description
0,10.0,Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.,https://cdn.akamai.steamstatic.com/steam/apps/10/page_bg_generated_v6b.jpg?t=1602535893,Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.,Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.
1,20.0,"One of the most popular online action games of all time, Team Fortress Classic features over nine character classes -- from Medic to Spy to Demolition Man -- enlisted in a unique style of online team warfare. Each character class possesses unique weapons, items, and abilities, as teams compete online in a variety of game play modes.",https://cdn.akamai.steamstatic.com/steam/apps/20/page_bg_generated_v6b.jpg?t=1579634708,"One of the most popular online action games of all time, Team Fortress Classic features over nine character classes -- from Medic to Spy to Demolition Man -- enlisted in a unique style of online team warfare. Each character class possesses unique weapons, items, and abilities, as teams compete online in a variety of game play modes.","One of the most popular online action games of all time, Team Fortress Classic features over nine character classes -- from Medic to Spy to Demolition Man -- enlisted in a unique style of online team warfare. Each character class possesses unique weapons, items, and abilities, as teams compete online in a variety of game play modes."
2,40.0,"Enjoy fast-paced multiplayer gaming with Deathmatch Classic (a.k.a. DMC). Valve's tribute to the work of id software, DMC invites players to grab their rocket launchers and put their reflexes to the test in a collection of futuristic settings.",https://cdn.akamai.steamstatic.com/steam/apps/40/page_bg_generated_v6b.jpg?t=1568752159,"Enjoy fast-paced multiplayer gaming with Deathmatch Classic (a.k.a. DMC). Valve's tribute to the work of id software, DMC invites players to grab their rocket launchers and put their reflexes to the test in a collection of futuristic settings.","Enjoy fast-paced multiplayer gaming with Deathmatch Classic (a.k.a. DMC). Valve's tribute to the work of id software, DMC invites players to grab their rocket launchers and put their reflexes to the test in a collection of futuristic settings."
3,360.0,"Half-Life Deathmatch: Source is a recreation of the first multiplayer game set in the Half-Life universe. Features all the classic weapons and most-played maps, now running on the Source engine.",https://cdn.akamai.steamstatic.com/steam/apps/360/page_bg_generated_v6b.jpg?t=1567240329,"Half-Life Deathmatch: Source is a recreation of the first multiplayer game set in the Half-Life universe. Features all the classic weapons and most-played maps, now running on the Source engine.","Half-Life Deathmatch: Source is a recreation of the first multiplayer game set in the Half-Life universe. Features all the classic weapons and most-played maps, now running on the Source engine."


Just by reading through the game description, the games does not seem to be similar to each other except being multiplayer games. Let us explore other features. 

### `genre`, `categories`, `languages` 

In [38]:
# see the information on genre, categories and languages
sql_query = """
SELECT genre.steam_appid, genre.genre, categories.categories_description, language.languages
FROM genre
INNER JOIN categories
  ON genre.steam_appid = categories.steam_appid
INNER JOIN language
  ON genre.steam_appid = language.steam_appid
WHERE genre.steam_appid IN (SELECT steam_appid FROM main
                        WHERE (name = 'Counter-Strike') OR (name = 'Team Fortress Classic') OR 
                                (name = 'Deathmatch Classic') OR (name = 'Half-Life Deathmatch: Source'))
"""

pd.read_sql(sql_query, con)

,steam_appid,genre,categories_description,languages
0,10.0,Action,"Multi-player,PvP,Online PvP,Shared/Split Screen PvP,Valve Anti-Cheat enabled","english,french,german,italian,spanish-spain,simplifiedchinese,traditionalchinese,korean"
1,20.0,Action,"Multi-player,PvP,Online PvP,Shared/Split Screen PvP,Valve Anti-Cheat enabled,Remote Play Together","english,french,german,italian,spanish-spain,korean,russian,simplifiedchinese,traditionalchinese"
2,40.0,Action,"Multi-player,PvP,Online PvP,Shared/Split Screen PvP,Valve Anti-Cheat enabled,Remote Play Together","english,french,german,italian,spanish-spain,korean,russian,simplifiedchinese,traditionalchinese"
3,360.0,Action,"Multi-player,Valve Anti-Cheat enabled",english


- Looking at the genres, we see that all 4 games are `Action` games. 
- The common categories for the 4 games are `Multi-player` and `Valve Anti-Cheat enabled`
- All 4 games are supported in English

### `tag`

In [39]:
# see the information from tag
sql_query = """
SELECT * 
FROM tag
WHERE steam_appid IN (SELECT steam_appid FROM main
                        WHERE (name = 'Counter-Strike') OR (name = 'Team Fortress Classic') 
                                OR (name = 'Deathmatch Classic') OR (name = 'Half-Life Deathmatch: Source'))
"""

df_tag_extract = pd.read_sql(sql_query, con)

In [40]:
# create list_tag_temp
list_tag_temp = list(df_tag_extract.columns)
list_tag_temp.remove('steam_appid')

# extract columns
for col in list_tag_temp:
    df_tag_extract[col] = df_tag_extract[col].apply(lambda x: 0 if x == -9999 else x)

# create column
df_tag_extract['combined'] = df_tag_extract.loc[:, list_tag_temp].to_dict(orient='records')

In [41]:
# remove value with zero from combined
df_tag_extract['combined'] = df_tag_extract['combined'].apply(lambda x: {k:v for k,v in x.items() if v!=0})

In [42]:
# see table
df_tag_extract[['steam_appid','combined']]

,steam_appid,combined
0,10.0,"{'Action': 5379.0, 'FPS': 4801.0, 'Multiplayer': 3362.0, 'Shooter': 3327.0, 'Classic': 2758.0, 'Team-Based': 1844.0, 'First-Person': 1692.0, 'Competitive': 1588.0, 'Tactical': 1323.0, '1990's': 1181.0, 'e-sports': 1173.0, 'PvP': 865.0, 'Old School': 751.0, 'Military': 623.0, 'Strategy': 604.0, 'Survival': 296.0, 'Score Attack': 285.0, '1980s': 256.0, 'Assassin': 223.0, 'Violent': 65.0}"
1,20.0,"{'Action': 745.0, 'FPS': 306.0, 'Multiplayer': 257.0, 'Shooter': 206.0, 'Classic': 232.0, 'Team-Based': 188.0, 'First-Person': 169.0, 'Competitive': 68.0, '1990's': 132.0, 'Old School': 106.0, 'Violent': 45.0, 'Hero Shooter': 213.0, 'Class-Based': 181.0, 'Co-op': 89.0, 'Fast-Paced': 61.0, 'Retro': 55.0, 'Online Co-Op': 51.0, 'Mod': 36.0, 'Funny': 35.0, 'Remake': 35.0}"
2,40.0,"{'Action': 629.0, 'FPS': 139.0, 'Multiplayer': 96.0, 'Shooter': 94.0, 'Classic': 107.0, 'First-Person': 70.0, 'Competitive': 23.0, '1990's': 8.0, 'Old School': 33.0, 'Co-op': 13.0, 'Fast-Paced': 15.0, 'Retro': 14.0, 'Difficult': 12.0, 'Arena Shooter': 44.0, 'Sci-fi': 33.0, 'Gore': 14.0}"
3,360.0,"{'Action': 241.0, 'FPS': 158.0, 'Multiplayer': 121.0, 'Shooter': 95.0, 'Classic': 69.0, 'First-Person': 74.0, 'Competitive': 10.0, 'Fast-Paced': 9.0, 'Remake': 24.0, 'Singleplayer': 6.0, 'Sci-fi': 85.0, 'Adventure': 15.0, 'Atmospheric': 8.0, 'Horror': 10.0}"


In [43]:
# create new dataframe to store the columns that these games have
df_tag_extract_rev = df_tag_extract[['steam_appid']].join(pd.DataFrame(df_tag_extract['combined'].tolist()))

In [44]:
# see new dataframe
df_tag_extract_rev

,steam_appid,Action,FPS,Multiplayer,Shooter,Classic,Team-Based,First-Person,Competitive,Tactical,1990's,e-sports,PvP,Old School,Military,Strategy,Survival,Score Attack,1980s,Assassin,Violent,Hero Shooter,Class-Based,Co-op,Fast-Paced,Retro,Online Co-Op,Mod,Funny,Remake,Difficult,Arena Shooter,Sci-fi,Gore,Singleplayer,Adventure,Atmospheric,Horror
0,10.0,5379.0,4801.0,3362.0,3327.0,2758.0,1844.0,1692.0,1588.0,1323.0,1181.0,1173.0,865.0,751.0,623.0,604.0,296.0,285.0,256.0,223.0,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20.0,745.0,306.0,257.0,206.0,232.0,188.0,169.0,68.0,NaN,132.0,NaN,NaN,106.0,NaN,NaN,NaN,NaN,NaN,NaN,45.0,213.0,181.0,89.0,61.0,55.0,51.0,36.0,35.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40.0,629.0,139.0,96.0,94.0,107.0,NaN,70.0,23.0,NaN,8.0,NaN,NaN,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,15.0,14.0,NaN,NaN,NaN,NaN,12.0,44.0,33.0,14.0,NaN,NaN,NaN,NaN
3,360.0,241.0,158.0,121.0,95.0,69.0,NaN,74.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,24.0,NaN,NaN,85.0,NaN,6.0,15.0,8.0,10.0


We see that users of the games have selected the tags, and comparing between the 4 games, they are common in being
- `Action`
- `FPS`
- `Multiplayer`
- `Shooter`
- `Classic`
- `First-Person`
- `Competitive`

### `price`, `media`, `requirements`

In [45]:
# see the information from price and media and requirements
sql_query = """
SELECT media.*, price.*, requirements.linux_platform, requirements.mac_platform, requirements.windows_platform
FROM media
INNER JOIN price
  ON media.steam_appid == price.steam_appid
INNER JOIN requirements
  ON media.steam_appid == requirements.steam_appid
WHERE media.steam_appid IN (SELECT steam_appid FROM main
                        WHERE (name = 'Counter-Strike') OR (name = 'Team Fortress Classic') OR (name = 'Deathmatch Classic') OR (name = 'Half-Life Deathmatch: Source'))
"""

pd.read_sql(sql_query, con)

,steam_appid,header_image,has_movies,num_movies,has_screenshots,num_screenshots,steam_appid,is_free,currency,initial_price,final_price,discount_percent,linux_platform,mac_platform,windows_platform
0,10.0,https://cdn.akamai.steamstatic.com/steam/apps/10/header.jpg?t=1602535893,0,0,1,13,10.0,0,EUR,819.0,819.0,0.0,1,1,1
1,20.0,https://cdn.akamai.steamstatic.com/steam/apps/20/header.jpg?t=1579634708,0,0,1,5,20.0,0,SGD,525.0,525.0,0.0,1,1,1
2,40.0,https://cdn.akamai.steamstatic.com/steam/apps/40/header.jpg?t=1568752159,0,0,1,4,40.0,0,SGD,525.0,525.0,0.0,1,1,1
3,360.0,https://cdn.akamai.steamstatic.com/steam/apps/360/header.jpg?t=1567240329,0,0,1,5,360.0,0,SGD,1000.0,1000.0,0.0,1,1,1


We see that for all 4 games, they have used 1 screenshot to promote their game on steam. 
We see that all 4 games have not free, but the prices are around SGD 12 or less. 
All 4 games are also playable on all 3 platforms. 

### `statistics`, `support_info`

In [46]:
# see the information from statistics and support info
sql_query = """
SELECT statistics.*, support_info.support_url, support_info.support_email
FROM statistics
INNER JOIN support_info
  ON statistics.steam_appid == support_info.steam_appid
WHERE statistics.steam_appid IN (SELECT steam_appid FROM main
                            WHERE (name = 'Counter-Strike') OR (name = 'Team Fortress Classic') 
                                    OR (name = 'Deathmatch Classic') OR (name = 'Half-Life Deathmatch: Source'))
"""

pd.read_sql(sql_query, con)

,steam_appid,average_2weeks,average_forever,ccu,median_2weeks,median_forever,negative,positive,userscore,min_owners,max_owners,review_score,review_percent,support_url,support_email
0,10.0,212.0,8690.0,16837.0,116.0,239.0,4944.0,193192.0,0.0,10000000,20000000,188248.0,3.966456,http://steamcommunity.com/app/10,
1,20.0,0.0,2752.0,77.0,0.0,16.0,896.0,5416.0,0.0,5000000,10000000,4520.0,0.095238,,
2,40.0,0.0,5083.0,5.0,0.0,7.0,412.0,1854.0,0.0,5000000,10000000,1442.0,0.030383,,
3,360.0,0.0,1740.0,15.0,0.0,25.0,776.0,2344.0,0.0,5000000,10000000,1568.0,0.033038,,


We see that there is not many recent users playing `Team Fortress Classic`, `Deathmatch Classic` and `Half-Life Deathmatch: Source`.

We also see that the number of reviews (positive or negative) does not seem to have a relationship with playtime. 

## Conclusion

We see that the recommender is not amazingly accurate, but it serves as a start for people who to explore other games that they could be interested in. 

We will store the cosine similarity calculation and use it to build a recommender application. 

### Save dataframe

We will save (export) the required dataframes to output files. We will use the exported files to build a recommender on streamlit. 

We can run the below codes to save the dataframe. For our case, we will be building a simple recommender so we will not run the below codes. 

In [48]:
"""# create new dataframe to store
sql_query = '''
SELECT main.steam_appid, main.name, media.header_image
FROM main
INNER JOIN media
    ON main.steam_appid = media.steam_appid
'''

df_store = pd.read_sql(sql_query, con)"""

In [75]:
"""# see shape of cos_sim_two
cos_sim_two.shape"""

(49015, 49015)

Due to the large number of columns in `cos_sim_two`, we will not be able to store the file in the database (and also in `.csv`).

As an alternative, we will store the dataframe that was used to calculate cosine similarity.

In [87]:
"""pd.DataFrame(df_model_two).shape"""

(49015, 3103)

As the number of columns for the dataframe that will be used to calculate cosine similarity has too many columns to be stored in database ([link](https://www.ibm.com/support/pages/filenet-content-manager-ms-sql-server-database-table-limitations-columns-and-row-size)), we will store the files in `.csv` format. 

In [91]:
"""# storing the df
pd.DataFrame(df_model_two).to_csv("../data/model_data.csv", index=False)
df_store.to_csv("../data/name_data.csv", index=False)"""